In [6]:
import requests
import mysql.connector
from datetime import datetime  
import json

In [7]:
# 创建数据库连接
db = mysql.connector.connect(
    host="localhost",  # MySQL服务器地址
    user="root",   # 用户名
    password="",  # 密码
    database="soybean"  # 数据库名称
)

# 创建游标对象，用于执行SQL查询
cursor = db.cursor()

In [8]:
today_str = datetime.now().strftime("%Y-%m-%d")



cursor.execute("SELECT news_date,gpt_value FROM news WHERE news_date = '"+today_str+"'")
res = cursor.fetchall()     # fetchall() 获取所有记录

news_list = []
for x in res:
    news_list.append(list(x)) 

In [9]:
news_list

[[datetime.datetime(2024, 9, 8, 0, 0), 1.0],
 [datetime.datetime(2024, 9, 8, 0, 0), 0.5],
 [datetime.datetime(2024, 9, 8, 0, 0), 1.0],
 [datetime.datetime(2024, 9, 8, 0, 0), 0.5]]

In [10]:
from collections import defaultdict  
  
# 假设这是你的数据  
data = news_list
  
# 使用 defaultdict 来按日期分组并计数  
from datetime import datetime  
count_by_date = defaultdict(lambda: defaultdict(int))  
  
for date, value in data:  
    date_str = date.strftime('%Y-%m-%d')  # 将日期转换为字符串以便作为字典的键  
    count_by_date[date_str][value] += 1  
  
# 遍历每个日期，找出数量最多的值  
results = []  
for date_str, values in count_by_date.items():  
    max_count = max(values.values())  # 找到该日期下最多的数量  
    max_values = [k for k, v in values.items() if v == max_count]  # 找到所有数量最多的值  
      
    # 如果0和1的数量都等于最大值且都大于0.5的数量，则选择0.5  
    if '0' in max_values and '1' in max_values and (values['0'] == values['1'] > values.get('0.5', 0)):  
        most_common_value = '0.5'  
    else:  
        most_common_value = max_values[0]  # 如果不是特殊情况，就取第一个最大值  
  
    results.append((date_str, most_common_value))  
  
# 打印结果  
for date_str, most_common_value in results:  
    cursor.execute("SELECT *FROM news_statistics WHERE record_date = '"+today_str+"'")
    res = cursor.fetchall()     # fetchall() 获取所有记录
    
    if len(res) > 0 :
        continue;
    
    sql = "INSERT INTO news_statistics (record_date, trend_value) VALUES (%s, %s)"
    values = (date_str,most_common_value)
    cursor.execute(sql, values)
# 提交更改到数据库
db.commit()